# Marketing Assistant

Copyright 2025, Denis Rothman

**Goal:** This notebook serves as the practical runtime for the **Strategic Marketing Engine**, as architected in Chapter 9. It demonstrates how the generic, multi-domain "Glass Box" Context Engine can be seamlessly repurposed to solve a variety of real-world marketing challenges without any changes to its core code.

This notebook will:
* Connect to the Pinecone knowledge base populated by the `Data_Ingestion_Marketing.ipynb` script.
* Use the generic Control Deck templates to interact with the engine.
* Execute the marketing use cases




# I. Inititalization

## GitHub

In [ ]:
print("Downloading files from public repository...")

# The -f flag tells curl to fail on an error (like 404)
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch9/utils.py --output utils.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch9/helpers.py --output helpers.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch9/agents.py --output agents.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch9/registry.py.py --output registry.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch9/engine.py --output engine.py
# (You might want to add a check here to see if the files actually exist now)
print("✅ File download attempt finished!")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0    14    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (22) The requested URL returned error: 404
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0    14    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (22) The requested URL returned error: 404
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0    14    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (22) The requested URL returned error: 404
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent

In [ ]:
# This cell will be deleted when repo goes public
import glob

# glob.glob("/content/*.py") will return a list of matching files.
# If the list is not empty, it evaluates to True.
# If the list is empty (no .py files found), it evaluates to False.

if glob.glob("/content/*.py"):
    filesdownloaded = True
else:
    filesdownloaded = False

print(f"Are .py files present in /content? {filesdownloaded}")

import os
import glob
from google.colab import userdata

# --- Configuration ---
OWNER = "Denis2054"
REPO = "Context-Engineering"
BRANCH = "main"

FILES_TO_DOWNLOAD = [
    {"remote": "commons/ch9/utils.py", "local": "utils.py"},
    {"remote": "commons/ch9/helpers.py", "local": "helpers.py"},
    {"remote": "commons/ch9/agents.py", "local": "agents.py"},
    {"remote": "commons/ch9/registry.py", "local": "registry.py"},
    {"remote": "commons/ch9/engine.py", "local": "engine.py"}
]
# --- End Configuration ---
# Check if all target files already exist
all_files_exist = all(os.path.exists(f["local"]) for f in FILES_TO_DOWNLOAD)

if not all_files_exist:
    print("ℹ️ Files not found. Attempting download using private token...")

    try:
        token = userdata.get('GITHUB_TOKEN')
        if not token:
            print("🛑 Error: 'GITHUB_TOKEN' not found in Colab Secrets.")
        else:
            # Loop and download each file that is missing
            for f in FILES_TO_DOWNLOAD:
                if not os.path.exists(f["local"]):
                    # --- FIX: Corrected the typo 'httpsfs://' to 'https://' ---
                    api_url = f"https://api.github.com/repos/{OWNER}/{REPO}/contents/{f['remote']}?ref={BRANCH}"

                    # This is the !curl command, run from Python
                    # We use os.system to execute it
                    print(f"Downloading {f['local']}...")
                    command = (
                        f"curl -Lfs "
                        f"-H \"Authorization: Bearer {token}\" "
                        f"-H \"Accept: application/vnd.github.v3.raw\" "
                        f"\"{api_url}\" -o \"{f['local']}\""
                    )

                    # os.system returns 0 on success
                    if os.system(command) != 0:
                        print(f"🛑 Error downloading {f['local']}.")

            # Final check
            if all(os.path.exists(f["local"]) for f in FILES_TO_DOWNLOAD):
                print("✅ All files successfully downloaded.")
            else:
                print("🛑 Some files failed to download.")

    except Exception as e:
        print(f"An error occurred: {e}")
else:
    print("✅ Files already present. Skipping download.")



Are .py files present in /content? False
ℹ️ Files not found. Attempting download using private token...
✅ All files successfully downloaded.


## Installation and client setup

In [ ]:
#Installation and Client Setup

# Import the setup functions from your new utility file
import utils

# Run the installation
utils.install_dependencies()

# Initialize the OpenAI and Pinecone clients
client, pc = utils.initialize_clients()

🚀 Installing required packages...
✅ All packages installed successfully.

🔑 Initializing API clients...
   - OpenAI client initialized.
   - Pinecone client initialized.
✅ Clients initialized successfully.


## Context Engine library Import

In [ ]:
# 1. Import the hardened helper functions (LLM, Embeddings, Pinecone)
import helpers

# 2. Import the specialist agent functions (Librarian, Researcher, Writer)
import agents

# 3. Import the AGENT_TOOLKIT object that knows about all the agents
from registry import AGENT_TOOLKIT

# 4. Import the main context_engine function that orchestrates the entire process
from engine import context_engine

## Engine Room

In [ ]:
# === ENGINE ROOM: The Main Execution Function ===
# This function contains all the logic to run the engine.
# We define it here so our final cell can be very simple.

import logging
import pprint
from IPython.display import display, Markdown

# In Legal_Compliance_Assistant.ipynb (The "Engine Room" cell)

def execute_and_display(goal, config, client, pc, moderation_active):
    """
    Runs the context engine, now with an optional, two-stage moderation check.
    """
    # --- PRE-FLIGHT MODERATION CHECK (on user input) ---
    if moderation_active:
        print("--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---")
        moderation_report = helpers.helper_moderate_content(text_to_moderate=goal, client=client)

        print("Moderation Report:")
        pprint.pprint(moderation_report)

        if moderation_report["flagged"]:
            print("\n🛑 Goal failed pre-flight moderation. Execution halted.")
            return # Halt execution before calling the engine

    logging.info(f"******** Starting Engine for Goal: '{goal}' **********\\n")

    # 1. Run the Context Engine using the provided configuration
    result, trace = context_engine(
        goal,
        client=client,
        pc=pc,
        **config
    )

    # --- POST-FLIGHT MODERATION CHECK (on AI output) ---
    if result and moderation_active:
        print("\n--- [Safety Guardrail] Performing Post-Flight Moderation Check on Output ---")
        moderation_report = helpers.helper_moderate_content(text_to_moderate=result, client=client)

        print("Moderation Report:")
        pprint.pprint(moderation_report)

        if moderation_report["flagged"]:
            print("\n🛑 Generated output failed post-flight moderation and will be redacted.")
            result = "[Content flagged as potentially harmful by moderation policy and has been redacted.]"

    # 2. Display the Final Result
    print("\n--- FINAL OUTPUT ---")
    if result:
        display(Markdown(result))
    else:
        print(f"The engine failed to produce a result. Status: {trace.status}")

    # 3. Display the Technical Trace
    print("\\n\\n--- TECHNICAL TRACE (for the tech reader) ---")
    if trace:
        print(f"Trace Status: {trace.status}")
        print(f"Total Duration: {trace.duration:.2f} seconds")
        print("Execution Steps:")
        # --- THIS LINE IS TO CREATE THE 'pp' OBJECT ---
        pp = pprint.PrettyPrinter(indent=2)

        pp.pprint(trace.steps)

## Control Deck configuration

In [ ]:
# 1. Define all configuration variables for this run in a dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

#III.CONTROL DECKS

=== CONTROL DECK: Define Goal and Run Engine ===
This is the main interactive cell.
1. Change the 'goal' variable to your desired task.
2. Run this cell.


In [ ]:
#@title CONTROL DECK: Moderation
# 1. Define a simple, safe goal to test the moderation workflow.
goal = "Summarize the key points of the QuantumDrive"

# 2. Define the standard configuration.
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function with moderation explicitly activated.
execute_and_display(goal, config, client, pc, moderation_active=True)

--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---
Moderation Report:
{'categories': {'harassment': False,
                'harassment/threatening': False,
                'harassment_threatening': False,
                'hate': False,
                'hate/threatening': False,
                'hate_threatening': False,
                'illicit': False,
                'illicit/violent': False,
                'illicit_violent': False,
                'self-harm': False,
                'self-harm/instructions': False,
                'self-harm/intent': False,
                'self_harm': False,
                'self_harm_instructions': False,
                'self_harm_intent': False,
                'sexual': False,
                'sexual/minors': False,
                'sexual_minors': False,
                'violence': False,
                'violence/graphic': False,
                'violence_graphic': False},
 'flagged': False,
 'scores': {'harassment': 4

QuantumDrive Q-1 is a pro-focused NVMe 2.0 SSD over PCIe Gen 5 aimed at creators who want faster loads and renders. It comes in 2 TB, 4 TB, and 8 TB, with up to 7,500 MB/s reads and 7,000 MB/s writes, plus a graphene heat spreader to help prevent throttling under heavy workloads. You get AES-256 hardware encryption and the bundled DataWeaver backup/encryption suite, a 5-year limited warranty, and the 4 TB model is rated for 3,000 TBW. It’s being launched by the Project QuantumDrive team (the exact brand isn’t spelled out) and is in a Q1 pre-order phase on their site; shipping timelines aren’t listed. Heads-up on gaps: no details yet on controller, NAND, random IOPS, power use, pricing, or TBW for the 2 TB/8 TB models, and there aren’t third-party benchmarks—just the “End the Wait” marketing claims. Target use cases are video editing, 3D, and photo workflows, and it’s pitched as a big, single-step upgrade for creative workstations.

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 186.68 seconds
Execution Steps:
[ { 'agent': 'Librarian',
    'output': { 'blueprint_json': '{"scene_goal": "Summarize information '
                                  'quickly and casually.", "style_guide": "Use '
                                  'informal language. Keep it brief and '
                                  'engaging. Imagine explaining it to a '
                                  'friend.", "instruction": "Summarize the '
                                  'provided facts using the casual style '
                                  'guide."}'},
    'planned_input': { 'intent_query': 'Executive bullet-point summary: '
                                       'concise key points for a general '
                                       'audience with short headings and '
                                       'bullets; cover what it is, how it '
                                       'works (high

In [ ]:
#@title Product Marketing Copy Generation(Use Case 1)
goal = "Analyze the ChronoTech press release and summarize their core product messaging and value proposition. Please cite your sources."


# 2. Use the standard configuration
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


Definition
ChronoTech’s Chrono SSD Pro is a flagship PCIe Gen 4 solid-state drive designed for modern creators. It targets digital artists and content creators handling large media projects who require high‑throughput storage to eliminate workflow bottlenecks.

Function/Operation
The drive is built on a PCIe Gen 4 platform and is optimized for maximum read/write performance, delivering up to 7,300 MB/s sequential read speeds. A 4TB capacity option supports large project datasets and media libraries.

Key Findings/Impact
- Performance-first design on a proven PCIe Gen 4 platform, offering flagship specifications and a highly competitive price point for the 4TB model.
- Minimizes waiting and accelerates creative workflows so technology does not impede creativity.
- Available for purchase today.
- Evidence includes a measured 7,300 MB/s sequential read performance and validation from CEO Jane Doe, who stated that creators are “tired of waiting” and emphasized the pursuit of the “fastest possible read and write speeds.”

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 173.33 seconds
Execution Steps:
[ { 'agent': 'Researcher',
    'output': { 'answer_with_sources': 'Here is the most recent official '
                                       'ChronoTech press release announcing '
                                       'their core product (as available in '
                                       'the provided materials).\n'
                                       '\n'
                                       '(a) Press release text (key sections)\n'
                                       '\n'
                                       '- Headline: \n'
                                       '  ChronoTech Unveils the Chrono SSD '
                                       'Pro: Speed for the Modern Creator [1]\n'
                                       '\n'
                                       '- Subhead: \n'
                                       '  FOR IMMEDIATE RELEASE [1]\n'


In [ ]:
#@title Product Marketing Copy Generation(Use Case 2)


# 1. Define the Goal: A research query that requires a verifiable, cited answer.
#    - DOMAIN: Any knowledge-intensive field (e.g., legal, medical, financial).
#    - KEY CAPABILITY: Tests the high-fidelity `Researcher` agent and its ability
#      to retrieve text with `source` metadata and generate citations.
# goal = "[INSERT YOUR HIGH-FIDELITY RESEARCH GOAL HERE]"

## 1. Define the Goal: A research query that asks for a creative output.
goal = "Using the official product spec sheet, write a short marketing description for the new QuantumDrive Q-1. The description should be confident, aspirational, and focus on the benefits for creative professionals. Please cite your sources."



# 2. Use the standard configuration
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


QuantumDrive Q-1 is a seriously fast PCIe Gen5 NVMe 2.0 SSD—think up to 7,500 MB/s sequential reads and 7,000 MB/s writes. You can get it in 2TB, 4TB, or 8TB. The 4TB model is rated for 3,000 TBW, and there’s a 5-year limited warranty. It also packs a graphene heat spreader to keep temps in check and prevent throttling. No size/weight details were provided. Info comes from the official Product Specification Sheet: Project QuantumDrive — QuantumDrive Q-1.

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 171.66 seconds
Execution Steps:
[ { 'agent': 'Researcher',
    'output': { 'answer_with_sources': 'Official product specification sheet '
                                       'located\n'
                                       '- Title: Product Specification Sheet: '
                                       'Project QuantumDrive — QuantumDrive '
                                       'Q-1\n'
                                       '- URL: Not provided in the supplied '
                                       'materials (appears to be the '
                                       'manufacturer’s official spec sheet)\n'
                                       '\n'
                                       'Verified specifications (from the spec '
                                       'sheet)\n'
                                       '- Product type: Solid-State Drive '
                                       

In [ ]:
#@title Writing a brand pitch recommendation(Use case 3)

# 1. Define the Goal: A creative or factual task that is deliberately
#    outside the scope of the documents in the knowledge base.
#    - DOMAIN: Universal test applicable to any curated knowledge base.
#    - KEY CAPABILITY: Tests the `Researcher` agent's ability to report a
#      negative finding and the `Writer` agent's ability to handle it gracefully,
#      preventing hallucination.
# goal = "[INSERT YOUR OUT-OF-SCOPE GOAL HERE]"


# =CONTROL DECK 3: A persuasive pitch ===
goal = "Write a persuasive pitch on our brand tone and voice guide"


# 2. Use the same configuration dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


Quick take: a Brand Tone and Voice Guide makes everyone write the same way, so content ships faster, reads cleaner, and converts better—with less legal risk.

What it is
- A simple playbook for how your brand talks. The “Innovate Forward” version centers on Clarity, Confidence, and Aspiration, with do/don’t rules and a “forbidden language” list.

Why it matters (aka ROI levers)
- Consistency = trust: One voice from SEO pages to emails.
- Speed up production: Clear rules mean fewer guesses, faster drafts.
- Fewer edits: Shared checklists cut subjective feedback.
- Better outcomes: Stronger engagement and conversion thanks to unified problem → solution → proof messaging.
- Lower risk: No unguaranteed claims or competitor bashing = fewer compliance headaches.
- Faster onboarding: New writers ramp quickly with examples and templates.

“Yeah, but…” objections
- “Kills creativity?” Boundaries free you up to focus on ideas, not style.
- “Too much upkeep?” Keep it lightweight; review quarterly.
- “Does it scale?” Principles are channel-agnostic; applies across regions.

What to measure
- Consistency, speed, edit rounds, NPS/CSAT, SEO dwell/CTR, email open/CTR/conversions, compliance escalations, and time-to-first-approved-draft.

Mini case snaps
- SEO pillar: “Ultimate Guide to Video Editing Storage” uses scannable headings, active voice, and links to the QuantumDrive page.
- Email nurture: Problem (“Is slow storage holding you back?”) → Solution (“End the Wait”) → Proof (testimonial + 5-year warranty, clear pre-order CTA).

How to roll it out
- Codify rules and checklists, add channel tips (SEO vs. email), assign an owner, train the team, ship templates/snippets (“End the Wait”), and run a monthly KPI scorecard.

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 155.81 seconds
Execution Steps:
[ { 'agent': 'Librarian',
    'output': { 'blueprint_json': '{"scene_goal": "Summarize information '
                                  'quickly and casually.", "style_guide": "Use '
                                  'informal language. Keep it brief and '
                                  'engaging. Imagine explaining it to a '
                                  'friend.", "instruction": "Summarize the '
                                  'provided facts using the casual style '
                                  'guide."}'},
    'planned_input': { 'intent_query': 'Persuasive executive pitch for '
                                       'adopting and using a Brand Tone and '
                                       'Voice Guide—concise, benefit-led, '
                                       'objection-handling, proof-point '
                                       'driven, wi